# Tweet vs Vote Analysis

Compare what politicians tweet about versus how they actually vote on important topics.

In [ ]:
# Import libraries
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
import yaml

# Database connection
import sys
sys.path.insert(0, str(Path.cwd().parent / 'src'))
from xminer.io.db import engine
from sqlalchemy import text

print('✅ Libraries imported successfully')

In [ ]:
# Configuration
PARAMS_FILE = Path("../src/xminer/config/parameters.yml")

with PARAMS_FILE.open("r", encoding="utf-8") as f:
    params = yaml.safe_load(f) or {}

YEAR = int(params.get("year", 2025))
MONTH = int(params.get("month", 12))
YM = f"{YEAR:04d}{MONTH:02d}"

# Date range for tweets
START_DATE = "2025-12-01"
END_DATE = "2026-01-04"
STAND_TEXT = f"Zeitraum: 01. Dezember 2025 - 04. Januar 2026"

# Graphics directory
GRAPHICS_BASE_DIR = Path(params.get("graphics_base_dir", "../outputs"))
GRAPHICS_DIR = GRAPHICS_BASE_DIR / YM / "graphics" / "tweet_vs_vote"
GRAPHICS_DIR.mkdir(parents=True, exist_ok=True)

print(f"Period: {START_DATE} to {END_DATE}")
print(f"Output: {GRAPHICS_DIR}")

In [ ]:
# Party colors
PARTY_COLORS = {
    "CDU/CSU": "#000000",
    "SPD": "#E3000F",
    "GRÜNE": "#1AA64A",
    "BÜNDNIS 90/DIE GRÜNEN": "#1AA64A",
    "DIE LINKE.": "#BE3075",
    "FDP": "#FFED00",
    "AFD": "#009EE0",
    "BSW": "#009688",
}

def normalize_party(p: str) -> str:
    if p is None:
        return ""
    key = str(p).strip().upper()
    if key in {"CDU", "CSU"}:
        return "CDU/CSU"
    if key.startswith("GRÜN") or "GRUENE" in key or "B90" in key or "BÜNDNIS" in key or "BÜ90" in key:
        return "GRÜNE"
    if key in {"LINKE", "DIE LINKE", "DIE LINKE."}:
        return "DIE LINKE."
    return key

def get_party_color(party: str) -> str:
    normalized = normalize_party(party)
    return PARTY_COLORS.get(normalized, "#888888")

## Set Your Topic/Keyword Here

In [ ]:
# Change this to analyze different topics
TOPIC_KEYWORD = "Ukraine"  # Keyword to search in tweets

# Keyword mapping: some topics need broader search terms for votes
# Add your own mappings here for better results
KEYWORD_MAPPING = {
    "Ukraine": ["Ukraine", "russisch", "Russland"],  # Ukraine-related votes might mention Russia
    "Klima": ["Klima", "Klimaschutz"],
    "Mieten": ["Mieten", "Miet"],
    # Add more mappings as needed
}

# Get vote search terms (use mapping if available, otherwise use topic keyword)
VOTE_KEYWORDS = KEYWORD_MAPPING.get(TOPIC_KEYWORD, [TOPIC_KEYWORD])

print(f"Analyzing topic: {TOPIC_KEYWORD}")
print(f"Looking for votes about: {', '.join(VOTE_KEYWORDS)}")

## Step 1: Get Tweet Activity by Party

In [ ]:
# Query tweets mentioning the topic
tweet_query = f"""
SELECT 
    p.partei_kurz AS party,
    COUNT(DISTINCT t.username) as politicians_tweeting,
    COUNT(*) AS tweet_count,
    SUM(t.like_count) as total_likes,
    SUM(t.impression_count) as total_impressions
FROM public.tweets t
JOIN politicians_12_2025 p ON t.username = p.username
WHERE t.text ILIKE '%{TOPIC_KEYWORD}%'
  AND t.created_at >= '{START_DATE}'
  AND t.created_at <= '{END_DATE}'
GROUP BY p.partei_kurz
ORDER BY tweet_count DESC
"""

print(f"Querying tweets about '{TOPIC_KEYWORD}'...")

with engine.connect() as conn:
    df_tweets = pd.read_sql(text(tweet_query), conn)

# Use .loc to avoid chained assignment warning
df_tweets.loc[:, 'party_norm'] = df_tweets['party'].apply(normalize_party)

# Aggregate by normalized party
df_tweets_agg = (
    df_tweets.groupby('party_norm')
    .agg({
        'politicians_tweeting': 'sum',
        'tweet_count': 'sum',
        'total_likes': 'sum',
        'total_impressions': 'sum'
    })
    .reset_index()
    .sort_values('tweet_count', ascending=True)
)

print(f"✅ Found {df_tweets_agg['tweet_count'].sum():,} tweets from {df_tweets_agg['politicians_tweeting'].sum()} politicians\n")
df_tweets_agg

## Step 2: Get Voting Records by Party

In [ ]:
# Build WHERE clause for multiple keywords
keyword_conditions = " OR ".join([f"bv.vote_title ILIKE '%{kw}%'" for kw in VOTE_KEYWORDS])

# Query votes related to the topic
vote_query = f"""
SELECT 
    bv.vote_title,
    bv.vote_date,
    bv.wahlperiode,
    bv.sitzungnr,
    bv.abstimmnr,
    bv.fraktion_gruppe as party,
    COUNT(*) as total_votes,
    SUM(bv.ja) as ja_votes,
    SUM(bv.nein) as nein_votes,
    SUM(bv.enthaltung) as enthaltung_votes
FROM bundestag_votes bv
WHERE {keyword_conditions}
GROUP BY bv.vote_title, bv.vote_date, bv.wahlperiode, bv.sitzungnr, bv.abstimmnr, bv.fraktion_gruppe
ORDER BY bv.vote_date DESC
"""

print(f"Querying votes about: {', '.join(VOTE_KEYWORDS)}...")

with engine.connect() as conn:
    df_votes = pd.read_sql(text(vote_query), conn)

if df_votes.empty:
    print(f"⚠️  No votes found with any of these keywords: {', '.join(VOTE_KEYWORDS)}")
    print("\nAvailable votes:")
    list_query = "SELECT DISTINCT vote_title FROM bundestag_votes WHERE vote_title IS NOT NULL ORDER BY vote_title LIMIT 20"
    with engine.connect() as conn:
        available = pd.read_sql(text(list_query), conn)
    for title in available['vote_title']:
        print(f"  - {title}")
else:
    # Use .loc to avoid chained assignment warning
    df_votes.loc[:, 'party_norm'] = df_votes['party'].apply(normalize_party)
    
    print(f"✅ Found {len(df_votes['vote_title'].unique())} unique votes\n")
    print("Votes found:")
    for i, row in df_votes[['vote_title', 'vote_date']].drop_duplicates().iterrows():
        date_str = row['vote_date'].strftime('%Y-%m-%d') if pd.notna(row['vote_date']) else 'Unknown'
        print(f"  - {date_str}: {row['vote_title'][:80]}")
    
    df_votes

## Step 3: Aggregate Voting by Party

In [ ]:
if not df_votes.empty:
    # Aggregate votes by party across all related votes
    df_votes_agg = (
        df_votes.groupby('party_norm')
        .agg({
            'total_votes': 'sum',
            'ja_votes': 'sum',
            'nein_votes': 'sum',
            'enthaltung_votes': 'sum'
        })
        .reset_index()
    )
    
    # Calculate percentages
    df_votes_agg['ja_pct'] = (df_votes_agg['ja_votes'] / df_votes_agg['total_votes'] * 100).round(1)
    df_votes_agg['nein_pct'] = (df_votes_agg['nein_votes'] / df_votes_agg['total_votes'] * 100).round(1)
    
    # Determine dominant vote position
    df_votes_agg['position'] = df_votes_agg.apply(
        lambda row: 'Ja' if row['ja_votes'] > row['nein_votes'] else 'Nein', axis=1
    )
    
    df_votes_agg = df_votes_agg.sort_values('total_votes', ascending=True)
    
    print("\nVoting summary by party:")
    df_votes_agg

## Step 4: Create Combined Visualization

In [ ]:
if not df_votes.empty:
    # Get total politicians per party for percentage calculation (only those with X/Twitter accounts)
    total_politicians_query = """
    SELECT partei_kurz AS party, COUNT(*) as total_politicians
    FROM politicians_12_2025
    WHERE username IS NOT NULL
    GROUP BY partei_kurz
    """
    with engine.connect() as conn:
        df_total_pols = pd.read_sql(text(total_politicians_query), conn)
    
    df_total_pols.loc[:, 'party_norm'] = df_total_pols['party'].apply(normalize_party)
    df_total_pols_agg = df_total_pols.groupby('party_norm')['total_politicians'].sum().reset_index()
    
    # Merge tweet data with total politicians
    df_tweet_pct = pd.merge(
        df_tweets_agg[['party_norm', 'politicians_tweeting']],
        df_total_pols_agg,
        on='party_norm',
        how='left'
    )
    df_tweet_pct['tweet_pct'] = (df_tweet_pct['politicians_tweeting'] / df_tweet_pct['total_politicians'] * 100).round(1)
    
    # Calculate vote percentages
    df_vote_pct = df_votes_agg.copy()
    df_vote_pct['ja_pct'] = (df_vote_pct['ja_votes'] / df_vote_pct['total_votes'] * 100).round(1)
    
    # Merge tweet and vote percentages
    df_combined = pd.merge(
        df_tweet_pct[['party_norm', 'tweet_pct', 'politicians_tweeting', 'total_politicians']],
        df_vote_pct[['party_norm', 'ja_pct', 'ja_votes', 'nein_votes', 'total_votes']],
        on='party_norm',
        how='outer'
    ).fillna(0)
    
    df_combined = df_combined.sort_values('tweet_pct', ascending=False)  # Descending for vertical
    
    # Get party colors
    colors = [get_party_color(p) for p in df_combined['party_norm']]
    
    # Get unique vote titles for display
    vote_titles = df_votes[['vote_title', 'vote_date']].drop_duplicates().sort_values('vote_date', ascending=False)
    vote_titles_list = [
        f"<b>{row['vote_date'].strftime('%d.%m.%Y') if pd.notna(row['vote_date']) else 'N/A'}:</b> {row['vote_title']}"
        for _, row in vote_titles.iterrows()
    ]
    vote_titles_text = "<br><br>".join(vote_titles_list)  # Double line break for more spacing
    
    # Calculate dynamic height for vote titles section - much more space
    num_vote_lines = len(vote_titles_list)
    extra_height = max(200, num_vote_lines * 60)  # 60px per line for readability
    
    # Create vertical stacked charts (2 rows, 1 column)
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=(
            '<b>Wer tweetet darüber?</b>',
            '<b>Wer stimmt dafür?</b>'
        ),
        vertical_spacing=0.15,
        row_heights=[0.5, 0.5]
    )
    
    # Top chart: % of politicians tweeting
    fig.add_trace(
        go.Bar(
            x=df_combined['party_norm'],
            y=df_combined['tweet_pct'],
            marker_color=colors,
            text=[f"<b>{v:.0f}%</b>" for v in df_combined['tweet_pct']],
            textposition='outside',
            textfont=dict(color='white', size=16),
            name='',
            showlegend=False,
            hovertemplate=(
                "<b>%{x}</b><br>"
                "%{y:.1f}% der Abgeordneten mit X-Account tweeteten<br>"
                "<extra></extra>"
            )
        ),
        row=1, col=1
    )
    
    # Bottom chart: % voting Ja
    fig.add_trace(
        go.Bar(
            x=df_combined['party_norm'],
            y=df_combined['ja_pct'],
            marker_color=colors,
            text=[f"<b>{v:.0f}%</b>" for v in df_combined['ja_pct']],
            textposition='outside',
            textfont=dict(color='white', size=16),
            name='',
            showlegend=False,
            hovertemplate=(
                "<b>%{x}</b><br>"
                "%{y:.1f}% stimmten mit Ja<br>"
                "<extra></extra>"
            )
        ),
        row=2, col=1
    )
    
    # Main title
    title_text = f"<b>{TOPIC_KEYWORD}</b>: Tweets vs. Abstimmungen<br><sub style='font-size:0.7em;'>{STAND_TEXT}</sub>"
    
    # Total height calculation
    total_height = 1400 + extra_height
    
    fig.update_layout(
        title=dict(
            text=title_text, 
            x=0.5, 
            xanchor='center', 
            font=dict(size=28, color='white')
        ),
        plot_bgcolor='#1a1a1a',
        paper_bgcolor='#1a1a1a',
        font=dict(color='white', size=14, family='Arial'),
        height=total_height,
        showlegend=False,
        margin=dict(b=extra_height + 60, t=140, l=80, r=80)
    )
    
    # Add vote titles at the bottom with much larger font
    fig.add_annotation(
        text=f"<b style='font-size:18px'>Analysierte Abstimmungen:</b><br><br>{vote_titles_text}",
        xref="paper",
        yref="paper",
        x=0.5,
        y=-0.05,
        xanchor='center',
        yanchor='top',
        showarrow=False,
        font=dict(size=16, color='white', family='Arial'),
        align='left',
        bgcolor='rgba(30,30,30,0.95)',
        bordercolor='#888888',
        borderwidth=2,
        borderpad=25
    )
    
    # Update axes
    fig.update_yaxes(
        title_text="<b>% mit X-Account</b>", 
        gridcolor='#333333',
        range=[0, 105],
        title_font=dict(size=15),
        row=1, col=1
    )
    fig.update_yaxes(
        title_text="<b>% Ja-Stimmen</b>", 
        gridcolor='#333333',
        range=[0, 105],
        title_font=dict(size=15),
        row=2, col=1
    )
    fig.update_xaxes(gridcolor='#333333', tickfont=dict(size=17), tickangle=-45, row=1, col=1)
    fig.update_xaxes(gridcolor='#333333', tickfont=dict(size=17), tickangle=-45, row=2, col=1)
    
    # Save as mobile-friendly portrait format (1080x1920 for Instagram/Stories)
    output_file = GRAPHICS_DIR / f"{TOPIC_KEYWORD.lower()}_tweet_vs_vote.png"
    fig.write_image(output_file, width=1080, height=total_height, scale=2)
    
    print(f"\n✅ Saved: {output_file}")
    
    # Display
    fig.show()
else:
    print("⚠️  Skipping visualization - no votes found for this topic")

## Step 5: Summary Statistics

In [ ]:
if not df_votes.empty:
    print("=" * 80)
    print(f"Tweet vs. Vote Analysis: {TOPIC_KEYWORD}")
    print("=" * 80)
    print()
    print("TWEET ACTIVITY:")
    print(f"  Total tweets: {df_tweets_agg['tweet_count'].sum():,}")
    print(f"  Politicians tweeting: {df_tweets_agg['politicians_tweeting'].sum()}")
    print(f"  Top party: {df_tweets_agg.iloc[-1]['party_norm']} ({df_tweets_agg.iloc[-1]['tweet_count']:,} tweets)")
    print()
    print("VOTING BEHAVIOR:")
    print(f"  Total votes analyzed: {len(df_votes['vote_title'].unique())}")
    print(f"  Total member votes: {df_votes_agg['total_votes'].sum():,}")
    print()
    print("  By Party:")
    for _, row in df_votes_agg.sort_values('total_votes', ascending=False).iterrows():
        print(f"    {row['party_norm']}: {int(row['ja_votes'])} Ja / {int(row['nein_votes'])} Nein ({row['position']} majority)")
    print()
    print(f"✅ Visualization saved to: {output_file}")
    print(f"✅ Ready for social media posting!")